In [ ]:
from model import PointGen
import torch
# torch.manual_seed(20)

<h3>Dimensionality check</h3>
<p>
Model should generate a (1, 1024, 3) and contain about 40m parameters. It took around 200ms to process one image in my laptop.
</p>

In [ ]:
model = PointGen()
# print(model)
print('Number of parameters: ', sum(p.numel() for p in model.parameters()))
x = torch.rand(1, 3, 224, 224)
model.eval()
model(x)  # cold start
%time ys = model(x)
print([y.size() for y in ys])

<h3>Data loader checks</h3>

In [ ]:
from data.modelnet import ModelNet10
from data.visualize import visualize_points
import open3d as o3d

data = ModelNet10(points_to_sample=1024)

In [ ]:
print(data.image_files[0])
d = data[0]
visualize_points(d[-1])

<h3>Overfitting Experiments</h3>

<p>Here, we sample a small set of data and try to overfit.</p>

In [1]:
import torch
from data.modelnet import ModelNet10
data = ModelNet10(points_to_sample=1024)

In [9]:
print(len(data))
data_size=8
idx = torch.randperm(len(data))[:data_size].tolist()
print(idx)

47892
[4533, 561, 5372, 46006, 44445, 25356, 6663, 11273]


In [10]:
small_data = [data[i] for i in idx]
# coarse_points = torch.stack([d[1] for d in small_data])
detailed_points = torch.stack([d[1] for d in small_data])
images = torch.stack([d[0] for d in small_data])

<h4>Overfitting the coarse predictor</h4>
<p>Here, we try to overfit the coarse prediction layer</p>

In [11]:
print(images.shape, detailed_points.shape)
'''images = (images - images.mean(dim=[0, 2, 3], keepdim=True)) \
    / images.std(dim=[0, 2, 3], keepdim=True).add(1e-5)
print(images.mean(dim=[0,2,3]))
print(images.std(dim=[0,2,3]))'''


torch.Size([8, 3, 224, 224]) torch.Size([8, 1024, 3])


'images = (images - images.mean(dim=[0, 2, 3], keepdim=True))     / images.std(dim=[0, 2, 3], keepdim=True).add(1e-5)\nprint(images.mean(dim=[0,2,3]))\nprint(images.std(dim=[0,2,3]))'

In [12]:
from model.encoder import Encoder
from model.decoder_fold import CoarseDecoder, AttentionFoldNet
from model import PointGen
from model.loss import chamfer_distance, std_loss, surface_loss 
from torch.optim import Adam, SGD
import torch.nn as nn

In [13]:
net = PointGen(fold=AttentionFoldNet())
net.coarse_decoder.sphere_deform = True
#print(net)
opt = Adam(net.parameters(), lr=1e-2)
batch_size = 8
epochs = 300
for e in range(epochs):
    idx = torch.randperm(data_size).tolist()
    for i in range(0, data_size, batch_size):
        # yc = coarse_points[i:i+batch_size]
        yd = detailed_points[i:i+batch_size]
        x = images[i:i+batch_size]
        opt.zero_grad()
        # yc_pred, yd_pred = net(x)
        yc_pred = net(x)[-1]
        # lossc = chamfer_distance(yc_pred, yd)
        lossd = chamfer_distance(yc_pred, yd)
        (lossd + 0 * surface_loss(yc_pred)).backward()
        opt.step()
        print('Loss: ', lossd)

Loss:  tensor(0.0992, grad_fn=<AddBackward0>)
Loss:  tensor(0.0829, grad_fn=<AddBackward0>)
Loss:  tensor(0.0579, grad_fn=<AddBackward0>)
Loss:  tensor(0.0405, grad_fn=<AddBackward0>)
Loss:  tensor(0.0345, grad_fn=<AddBackward0>)
Loss:  tensor(0.0308, grad_fn=<AddBackward0>)
Loss:  tensor(0.0274, grad_fn=<AddBackward0>)
Loss:  tensor(0.0247, grad_fn=<AddBackward0>)
Loss:  tensor(0.0225, grad_fn=<AddBackward0>)
Loss:  tensor(0.0213, grad_fn=<AddBackward0>)
Loss:  tensor(0.0201, grad_fn=<AddBackward0>)
Loss:  tensor(0.0200, grad_fn=<AddBackward0>)
Loss:  tensor(0.0181, grad_fn=<AddBackward0>)
Loss:  tensor(0.0179, grad_fn=<AddBackward0>)
Loss:  tensor(0.0177, grad_fn=<AddBackward0>)
Loss:  tensor(0.0173, grad_fn=<AddBackward0>)
Loss:  tensor(0.0164, grad_fn=<AddBackward0>)
Loss:  tensor(0.0164, grad_fn=<AddBackward0>)
Loss:  tensor(0.0158, grad_fn=<AddBackward0>)
Loss:  tensor(0.0150, grad_fn=<AddBackward0>)
Loss:  tensor(0.0149, grad_fn=<AddBackward0>)
Loss:  tensor(0.0150, grad_fn=<Add

KeyboardInterrupt: 

In [14]:
net.eval()
for i in range(data_size):
    img = images[i].unsqueeze(0)
    pc = detailed_points[i].unsqueeze(0)
    pred = net(img)[1]
    print('{} Distance: {}'.format(i, chamfer_distance(pred, pc)))

0 Distance: 0.014501012861728668
1 Distance: 0.0315433070063591
2 Distance: 0.02078903280198574
3 Distance: 0.021441981196403503
4 Distance: 0.016096390783786774
5 Distance: 0.01464398019015789
6 Distance: 0.011692200787365437
7 Distance: 0.009760008193552494


In [ ]:
torch.save(net.state_dict(), 'overfit2')

In [20]:
from data.visualize import visualize_points
# visualize_points(data[0][1])
net.eval()
with torch.no_grad():
    pred = net(images[-3].unsqueeze(0))[1]
pred = pred.detach().contiguous().squeeze()
print(pred.shape)
visualize_points(pred)
visualize_points(detailed_points[-3])

torch.Size([1024, 3])


In [ ]:
max_coord = detailed_points[3].abs().max(-1)[0]
print(max_coord.mean(), max_coord.std())

In [ ]:
pcd_load = o3d.io.read_point_cloud("view0.ply")
pcd_load2 = o3d.io.read_point_cloud("view0_pred.ply")
o3d.visualization.draw_geometries([pcd_load])

In [ ]:

o3d.visualization.draw_geometries([pcd_load2])